In [ ]:
import requests
from bs4 import BeautifulSoup
import smtplib
import time
import pandas as pd

URL = 'https://www.ebay.ca/sch/i.html?_from=R40&_trksid=p4432023.m570.l1311&_nkw=omega+speedmaster&_sacat=0'

# Use a different User-Agent string
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299"
}


def get_product_data():
    try:
        page = requests.get(URL, headers=headers)
        page.raise_for_status()

        soup = BeautifulSoup(page.content, 'html.parser')

        product_titles = soup.find_all('h3', class_='s-item__title')
        product_prices = soup.find_all('span', class_='s-item__price')

        products_data = []

        for title, price in zip(product_titles, product_prices):
            product_data = {
                'Product Name': title.get_text().strip(),
                'Price': price.get_text().strip(),
            }
            products_data.append(product_data)

        return products_data

    except requests.exceptions.RequestException as req_exc:
        print(f"An error occurred while making the request: {req_exc}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def send_email(subject, body):
    # Replace with your email credentials
    sender_email = 'wackywoolies@gmail.com'
    sender_password = 'idogiveafuck'
    receiver_email = 'jojin80@@gmail.com'

    message = f'Subject: {subject}\n\n{body}'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, message)
    server.quit()

if __name__ == '__main__':
    target_price = 3800.0
    max_retries = 3
    retries = 0

    while retries < max_retries:
        print(f"Retrying ({retries+1}/{max_retries})...")
        products_data = get_product_data()

        if products_data:
            df = pd.DataFrame(products_data)

            # Save the data to an Excel file
            df.to_excel('ebay_products.xlsx', index=False)

            for product in products_data:
                print(product['Product Name'], product['Price'])
                price_str = product['Price'].replace('CA $', '').replace(',', '').strip()
                price = float(price_str)
                if price <= target_price:
                    subject = f"Price Alert: {product['Product Name']}"
                    body = f"The price has reached CA${price:.2f}. Buy now!"
                    send_email(subject, body)
                    break

            if price <= target_price:
                break
        else:
            print("Failed to fetch data. Retrying in 10 seconds...")
            # Wait for 10 seconds before retrying
            time.sleep(10)

        retries += 1


